# AstraForge Toolkit — Local API smoke test

Quick sanity checks for the published package against a locally running AstraForge API, plus the same quick-start snippets from the README.

Prereqs:
- Backend running at `http://localhost:8001` (e.g., `make backend-serve`)
- An API key (create via the in-app API Keys screen or `/api/api-keys/`)
- Package installed (`pip install -e ../astraforge-python-package` or from PyPI)
- For the DeepAgent example, set `OPENAI_API_KEY` (or configure your preferred model) so LangChain can call a chat model.


In [ ]:
# Install the local package in editable mode (run from the examples/ folder)
%pip install -e ..

In [ ]:
%pip install python-dotenv

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

BASE_URL = os.getenv("ASTRA_FORGE_API_URL")
API_KEY = os.getenv("ASTRA_FORGE_API_KEY")

In [3]:
import logging, sys

# Emit sandbox debug logs only for AstraForge backends
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s [%(name)s] %(message)s',
    stream=sys.stdout,
    force=True,
)
logging.getLogger('astraforge_toolkit').setLevel(logging.DEBUG)
logging.getLogger('astraforge.sandbox').setLevel(logging.DEBUG)
# Quiet noisy libraries
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)

In [4]:
# Enable sandbox debug logging
os.environ['ASTRA_FORGE_SANDBOX_DEBUG'] = '1'

### Create a sandbox session (no DeepAgent conversation)
Use the sandbox API directly without creating a DeepAgent conversation.


In [5]:
from astraforge_toolkit import DeepAgentClient

client = DeepAgentClient(base_url=BASE_URL, api_key=API_KEY)
sandbox_session = client.create_sandbox_session()
SANDBOX_SESSION_ID = sandbox_session.session_id
print(f"Sandbox session: {SANDBOX_SESSION_ID} (workspace: {sandbox_session.workspace_path})")

Sandbox session: f880696f-1c51-4ecd-b028-3d1119075d6a (workspace: /workspace)


### Upload and read sandbox files
Use the helper methods on `DeepAgentClient` to push a file into the sandbox and fetch it back.


In [ ]:
# Upload text (or bytes) to the sandbox workspace
upload_result = client.upload_file(
    SANDBOX_SESSION_ID,
    "/workspace/hello_from_notebook.txt",
    content="hello from the notebook!\n",
)
upload_result

In [ ]:
# Read the file back; set encoding=None to get raw bytes
downloaded = client.get_file_content(
    SANDBOX_SESSION_ID,
    "/workspace/hello_from_notebook.txt",
    encoding="utf-8",
)
print(downloaded)


### Create a sandbox-backed DeepAgent (README quick start)
This creates a DeepAgent conversation (separate from the standalone sandbox session above) and reuses its sandbox across tool calls.


In [ ]:
# Install a model provider helper for the example (skip if already installed)
%pip install langchain-openai

In [7]:
from deepagents import create_deep_agent

from langchain_openai import ChatOpenAI
from astraforge_toolkit import (
    SandboxBackend,
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
)

conv = client.create_conversation()
AGENT_THREAD_ID = conv.conversation_id
AGENT_SANDBOX_SESSION_ID = conv.sandbox_session_id
print(f"Using DeepAgent conversation {AGENT_THREAD_ID} and sandbox {AGENT_SANDBOX_SESSION_ID} for tool calls")

def backend_factory(rt):
    return SandboxBackend(
        rt,
        base_url=BASE_URL,
        api_key=API_KEY,
        session_id=AGENT_SANDBOX_SESSION_ID,
        # optional: session_params={"image": "astraforge/codex-cli:latest"},
    )

model = ChatOpenAI(model="gpt-4o-mini")  # or any chat model you prefer
tools = [
    sandbox_shell,
    sandbox_python_repl,
    sandbox_open_url_with_playwright,
    sandbox_view_image,
]

deep_agent = create_deep_agent(
    model=model,
    backend=backend_factory,
    tools=tools,
)

RUN_CONFIG = {
    "thread_id": AGENT_THREAD_ID,
    "configurable": {"sandbox_session_id": AGENT_SANDBOX_SESSION_ID},
}


Using DeepAgent conversation eacd793b-ae01-48f8-8372-956dc4ef3de4 and sandbox eacd793b-ae01-48f8-8372-956dc4ef3de4 for tool calls


In [ ]:
deep_agent.invoke(
    {"messages": [{"role": "user", "content": "What tools i have ?"}]},
    config=RUN_CONFIG,
)

# Reuse RUN_CONFIG for follow-up calls to keep the same sandbox session and thread ID.

In [8]:
result = deep_agent.invoke(
    {"messages": [{"role": "user", "content": "write a main.py that prints hello world"}]},
    config=RUN_CONFIG,
)
result

2025-12-03 16:40:26,090 DEBUG [astraforge_toolkit.backend] sandbox shell
2025-12-03 16:40:26,249 DEBUG [astraforge_toolkit.backend] sandbox write failed: path=/main.py exit=2 stderr='' stdout='sh: 1: cannot create /main.py: Permission denied'
2025-12-03 16:40:28,842 DEBUG [astraforge_toolkit.backend] sandbox shell
2025-12-03 16:40:29,017 DEBUG [astraforge_toolkit.backend] sandbox write failed: path=/main.py exit=2 stderr='' stdout='sh: 1: cannot create /main.py: Permission denied'
2025-12-03 16:40:30,965 DEBUG [astraforge_toolkit.backend] sandbox shell
2025-12-03 16:40:31,101 DEBUG [astraforge_toolkit.backend] sandbox write failed: path=/main.py exit=2 stderr='' stdout='sh: 1: cannot create /main.py: Permission denied'
2025-12-03 16:40:33,942 DEBUG [astraforge_toolkit.backend] sandbox shell


{'messages': [HumanMessage(content='write a main.py that prints hello world', additional_kwargs={}, response_metadata={}, id='7846305d-3bf5-48b0-a451-b911703cc6d5'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 4508, 'total_tokens': 4535, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b547601dbd', 'id': 'chatcmpl-CijFnZ3KJjaPoZjF2Wjx1f1awIHT9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--19498d5b-ecbb-4eba-9385-903cd6badc20-0', tool_calls=[{'name': 'write_file', 'args': {'file_path': '/main.py', 'content': "print('Hello, World!')"}, 'id': 'call_5yr6toPKVGGMxLHuv668xMdZ', 'type': 'tool_call'}], usage_me